In [26]:
import json, os, requests
from dateutil.relativedelta import relativedelta 
from datetime import datetime
import pandas as pd
pd.set_option('display.max_columns', 30)

In [27]:
current_datetime = datetime.now() - relativedelta(months=2)

formatted_datetime = current_datetime.strftime('%Y-%m-%d')


url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00.000' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59.000'&$limit=30000"
headers = {'X-App-Token': os.environ.get('CHICAGO_API_TOKEN')}

response = requests.get(url, headers=headers, verify=False)

data = response.json()


c:\Users\dterjeks\OneDrive - DXC Production\Scripts\Cubix_Data_Engineer_Repos\Cubix_Data_Engineer\venv\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.cityofchicago.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [28]:
taxi_trips = pd.DataFrame(data)

taxi_trips.head(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,a8efcfc881757b18c44f098d5c4c0d27769fbb6b,e721bbad6011a0096c3d8b0b6ad61b4762e4ac37079384...,2024-03-02T23:45:00.000,2024-03-03T00:00:00.000,532,0,6,3,13.55,0,0,0,13.55,Mobile,5 Star Taxi,41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",41.96581197,-87.655878786,"{'type': 'Point', 'coordinates': [-87.65587878...",NaN,NaN
1,a8f498d90c49f9175cb5e168ecf28cd6e630160e,9cd1266b95309c8284b2603dd68f0c3d2f708c3d1f4879...,2024-03-02T23:45:00.000,2024-03-02T23:45:00.000,725,3.21,76,NaN,11.5,4.25,0,5,21.25,Credit Card,Flash Cab,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",NaN,NaN,NaN,NaN,NaN
2,aa355372382d3af8832d755b87562c61784a5865,dd2844d5f13c91c2d50ff5092bacd0f339c5b3c17376b5...,2024-03-02T23:45:00.000,2024-03-02T23:45:00.000,5,0,6,6,25,5.1,0,0,30.6,Credit Card,City Service,41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",NaN,NaN
3,abf5ddaf7db3d46f9764478dfb4e4c1399b2a8ee,d744f003d8f56f6a8b53b97c0589575fa9a975e9d12f66...,2024-03-02T23:45:00.000,2024-03-03T00:15:00.000,1644,8.16,39,47,25.25,0,0,0,25.25,Prcard,Flash Cab,41.808916283,-87.596183344,"{'type': 'Point', 'coordinates': [-87.59618334...",41.728182061,-87.596475596,"{'type': 'Point', 'coordinates': [-87.59647559...",NaN,NaN
4,af9e65df2212b472aaf0db9b5e01377274644b42,8dbaf4164524af926ddeab5dd084183e585f4cb8cc47fe...,2024-03-02T23:45:00.000,2024-03-03T00:00:00.000,956,8.86,32,77,23.75,0,0,0,23.75,Prcard,Medallion Leasin,41.878865584,-87.625192142,"{'type': 'Point', 'coordinates': [-87.62519214...",41.9867118,-87.663416405,"{'type': 'Point', 'coordinates': [-87.66341640...",NaN,NaN


In [29]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11394 entries, 0 to 11393
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     11394 non-null  object
 1   taxi_id                     11394 non-null  object
 2   trip_start_timestamp        11394 non-null  object
 3   trip_end_timestamp          11394 non-null  object
 4   trip_seconds                11390 non-null  object
 5   trip_miles                  11394 non-null  object
 6   pickup_community_area       11044 non-null  object
 7   dropoff_community_area      10285 non-null  object
 8   fare                        11352 non-null  object
 9   tips                        11352 non-null  object
 10  tolls                       11352 non-null  object
 11  extras                      11352 non-null  object
 12  trip_total                  11352 non-null  object
 13  payment_type                11394 non-null  ob

In [30]:
# lássuk miből mennyi van
taxi_trips.describe()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
count,11394,11394,11394,11394,11390,11394,11044,10285,11352,11352,11352,11352,11352,11394,11394,11044,11044,11044,10321,10321,10321,2720,2599
unique,11394,1743,96,112,2677,2065,77,77,1061,835,14,159,1843,7,30,128,128,128,141,141,141,51,64
top,a8efcfc881757b18c44f098d5c4c0d27769fbb6b,d40dae7ea46d61abca67eb53b157fe9cf0b485cca6dce1...,2024-03-02T19:00:00.000,2024-03-02T19:00:00.000,0,0,8,8,9,0,0,0,3.25,Credit Card,Flash Cab,41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",17031980000,17031320100
freq,1,32,241,242,236,1267,2524,2512,472,6349,11309,7320,251,3597,2881,1666,1666,1666,1417,1417,1417,740,361


In [31]:
# 5 példa ahol a viteldíj hiányzik
taxi_trips[taxi_trips['fare'].isna()].sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
1933,18cbb9c86f2303890cc5f07f83ad0a14fac1df6e,f3af8002f02eb5eea68aa4cba00200f124550bb4759054...,2024-03-02T20:15:00.000,2024-03-02T21:00:00.000,2102,17.06,76,8,NaN,NaN,NaN,NaN,NaN,Cash,Taxicab Insurance Agency Llc,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.900265687,-87.63210922,"{'type': 'Point', 'coordinates': [-87.63210921...",17031980000,17031081000
1156,b910527f368c3711350f191ecba31351e90d5b96,a1d450978c238cda3a675fe514b22ccdade956e95f9bd6...,2024-03-02T21:30:00.000,2024-03-02T21:45:00.000,207,0.76,32,32,NaN,NaN,NaN,NaN,NaN,Cash,Blue Ribbon Taxi Association,41.880994471,-87.632746489,"{'type': 'Point', 'coordinates': [-87.63274648...",41.880994471,-87.632746489,"{'type': 'Point', 'coordinates': [-87.63274648...",17031839100,17031839100
2593,2d09e91efdbc1e42abc2d5167345d6c0f97bc9f0,ed7090d32800eec667900e884a935a298e55efaaaea9d6...,2024-03-02T19:15:00.000,2024-03-02T19:30:00.000,403,2.68,6,6,NaN,NaN,NaN,NaN,NaN,Cash,City Service,41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",NaN,NaN
127,09e398f5af8109fdc3f25063cef90cc772e9210c,3e5af3b3cc4163e517f1e511970533751db25e2dd44471...,2024-03-02T23:30:00.000,2024-03-02T23:45:00.000,867,7.81,41,28,NaN,NaN,NaN,NaN,NaN,Mobile,Chicago Independents,41.794090253,-87.592310855,"{'type': 'Point', 'coordinates': [-87.59231085...",41.874005383,-87.66351755,"{'type': 'Point', 'coordinates': [-87.66351754...",NaN,NaN
10537,c2cbe07feed359f5d9b8d54eef7b51a3a8f5e42b,8b8d35663133904892cc1e87911f55ed0bdf9a9bd1093c...,2024-03-02T05:45:00.000,2024-03-02T06:00:00.000,1432,13.31,3,76,NaN,NaN,NaN,NaN,NaN,Mobile,Sun Taxi,41.96581197,-87.655878786,"{'type': 'Point', 'coordinates': [-87.65587878...",41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",NaN,NaN


### Transformation: deal with NaN values

In [32]:
# oszlopok eltűntetése
taxi_trips.drop(['pickup_census_tract','dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True) 

In [33]:
taxi_trips.dropna(inplace=True)

### Transformation: renaming


In [34]:
taxi_trips.rename(columns={
    'pickup_community_area': 'pickup_community_area_id',
    'dropoff_community_area': 'dropoff_community_area_id'},
    inplace=True
)

### Transformation: create helper column for weather

In [35]:
taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp'])

In [36]:
# azért hogy a weather table-t joinolni tudjuk fel kell kerekítenünk csak órára
taxi_trips["datetime_for_weather"] = taxi_trips['trip_start_timestamp'].dt.floor('h')

#### check joining the trips and the weather data

In [37]:
# Get weather data amd transform

current_datetime = datetime.now() - relativedelta(months=2)

formatted_datetime = current_datetime.strftime('%Y-%m-%d')

url = "https://archive-api.open-meteo.com/v1/era5"
date = '2024-04-01'
params={
    'latitude': 41.85,
    'longitude': -87.65,
    'start_date': formatted_datetime,
    'end_date': formatted_datetime,
    'hourly': 'temperature_2m,wind_speed_10m,precipitation,rain'
}

response = requests.get(url, params=params)

weather_data = response.json()

weather_data_filtered = {
'datetime': weather_data['hourly']['time'],
'temperature': weather_data['hourly']['temperature_2m'],
'wind_speed': weather_data['hourly']['wind_speed_10m'],
'precipitation': weather_data['hourly']['precipitation'],
'rain': weather_data['hourly']['rain']
}


weather_df = pd.DataFrame(weather_data_filtered)

weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

In [38]:
taxi_trips_with_weather = taxi_trips.merge(weather_df, left_on='datetime_for_weather', right_on='datetime')

In [39]:
taxi_trips_with_weather.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,datetime,temperature,wind_speed,precipitation,rain
0,a8efcfc881757b18c44f098d5c4c0d27769fbb6b,e721bbad6011a0096c3d8b0b6ad61b4762e4ac37079384...,2024-03-02 23:45:00,2024-03-03T00:00:00.000,532,0,6,3,13.55,0,0,0,13.55,Mobile,5 Star Taxi,41.944226601,-87.655998182,41.96581197,-87.655878786,2024-03-02 23:00:00,2024-03-02 23:00:00,11.8,16.5,0.0,0.0
1,aa355372382d3af8832d755b87562c61784a5865,dd2844d5f13c91c2d50ff5092bacd0f339c5b3c17376b5...,2024-03-02 23:45:00,2024-03-02T23:45:00.000,5,0,6,6,25,5.1,0,0,30.6,Credit Card,City Service,41.944226601,-87.655998182,41.944226601,-87.655998182,2024-03-02 23:00:00,2024-03-02 23:00:00,11.8,16.5,0.0,0.0
2,abf5ddaf7db3d46f9764478dfb4e4c1399b2a8ee,d744f003d8f56f6a8b53b97c0589575fa9a975e9d12f66...,2024-03-02 23:45:00,2024-03-03T00:15:00.000,1644,8.16,39,47,25.25,0,0,0,25.25,Prcard,Flash Cab,41.808916283,-87.596183344,41.728182061,-87.596475596,2024-03-02 23:00:00,2024-03-02 23:00:00,11.8,16.5,0.0,0.0
3,af9e65df2212b472aaf0db9b5e01377274644b42,8dbaf4164524af926ddeab5dd084183e585f4cb8cc47fe...,2024-03-02 23:45:00,2024-03-03T00:00:00.000,956,8.86,32,77,23.75,0,0,0,23.75,Prcard,Medallion Leasin,41.878865584,-87.625192142,41.9867118,-87.663416405,2024-03-02 23:00:00,2024-03-02 23:00:00,11.8,16.5,0.0,0.0
4,b35283187973e2a73090acc5701debe201ac47af,5cdde36a39ded2651da1686c7813baf589dac6eb873894...,2024-03-02 23:45:00,2024-03-03T00:00:00.000,923,5.16,28,6,17.52,4.42,0,0,21.94,Mobile,Tac - Yellow Cab Association,41.874005383,-87.66351755,41.944226601,-87.655998182,2024-03-02 23:00:00,2024-03-02 23:00:00,11.8,16.5,0.0,0.0


### Transformation: Data type conversions

In [40]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10135 entries, 0 to 11393
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     10135 non-null  object        
 1   taxi_id                     10135 non-null  object        
 2   trip_start_timestamp        10135 non-null  datetime64[ns]
 3   trip_end_timestamp          10135 non-null  object        
 4   trip_seconds                10135 non-null  object        
 5   trip_miles                  10135 non-null  object        
 6   pickup_community_area_id    10135 non-null  object        
 7   dropoff_community_area_id   10135 non-null  object        
 8   fare                        10135 non-null  object        
 9   tips                        10135 non-null  object        
 10  tolls                       10135 non-null  object        
 11  extras                      10135 non-null  object        


In [41]:
taxi_trips.head(1)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,a8efcfc881757b18c44f098d5c4c0d27769fbb6b,e721bbad6011a0096c3d8b0b6ad61b4762e4ac37079384...,2024-03-02 23:45:00,2024-03-03T00:00:00.000,532,0,6,3,13.55,0,0,0,13.55,Mobile,5 Star Taxi,41.944226601,-87.655998182,41.96581197,-87.655878786,2024-03-02 23:00:00


In [42]:
data_types = {
    'trip_end_timestamp': 'datetime64[ns]',
    'trip_seconds': 'int32',
    'trip_miles': 'float',
    'pickup_community_area_id': 'int8',
    'dropoff_community_area_id': 'int8',
    'fare': 'float',
    'tips': 'float',
    'tolls': 'float',
    'extras': 'float',
    'trip_total': 'float',
}

taxi_trips = taxi_trips.astype(data_types)

In [43]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10135 entries, 0 to 11393
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     10135 non-null  object        
 1   taxi_id                     10135 non-null  object        
 2   trip_start_timestamp        10135 non-null  datetime64[ns]
 3   trip_end_timestamp          10135 non-null  datetime64[ns]
 4   trip_seconds                10135 non-null  int32         
 5   trip_miles                  10135 non-null  float64       
 6   pickup_community_area_id    10135 non-null  int8          
 7   dropoff_community_area_id   10135 non-null  int8          
 8   fare                        10135 non-null  float64       
 9   tips                        10135 non-null  float64       
 10  tolls                       10135 non-null  float64       
 11  extras                      10135 non-null  float64       


### Memory usage

In [44]:
# Raw: 19038200
taxi_trips.memory_usage(deep=True).sum()

7287951

### Sanity checks

In [45]:
taxi_trips.describe()

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,datetime_for_weather
count,10135,10135,10135.000000,10135.000000,10135.000000,10135.000000,10135.000000,10135.000000,10135.000000,10135.000000,10135.000000,10135
mean,2024-03-02 14:43:11.928959232,2024-03-02 15:01:53.576714240,1116.731426,5.963553,32.287025,25.119191,20.652118,2.229312,0.003148,1.203252,24.247130,2024-03-02 14:20:35.757276672
min,2024-03-02 00:00:00,2024-03-02 00:00:00,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-03-02 00:00:00
25%,2024-03-02 11:30:00,2024-03-02 11:45:00,491.000000,0.950000,8.000000,8.000000,8.750000,0.000000,0.000000,0.000000,10.580000,2024-03-02 11:00:00
50%,2024-03-02 15:15:00,2024-03-02 15:45:00,920.000000,3.040000,28.000000,24.000000,15.090000,0.000000,0.000000,0.000000,17.750000,2024-03-02 15:00:00
75%,2024-03-02 18:45:00,2024-03-02 19:15:00,1535.000000,10.855000,49.000000,33.000000,30.750000,3.015000,0.000000,1.000000,32.000000,2024-03-02 18:00:00
max,2024-03-02 23:45:00,2024-03-03 15:15:00,71290.000000,79.320000,77.000000,77.000000,600.000000,50.000000,5.000000,102.500000,600.000000,2024-03-02 23:00:00
std,NaN,NaN,1550.157376,6.277997,25.590847,20.827205,16.011904,3.545593,0.103953,3.363568,18.782508,NaN


In [46]:
taxi_trips.nlargest(10, 'fare')

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
8014,10ce2f8e71724ff6ddfc522fca311cb5254d1252,ea214932102d54ad8cdbf0e0880a3e7a0902af54459971...,2024-03-02 12:15:00,2024-03-02 12:15:00,60,0.00,6,6,600.00,0.0,0.0,0.0,600.00,No Charge,Taxi Affiliation Services,41.944226601,-87.655998182,41.944226601,-87.655998182,2024-03-02 12:00:00
7915,a3c258ddd7ee3e2adfe3567a5aba69647b2e6365,a030e6189cee598b516978caa72ad33839a0bccece8353...,2024-03-02 12:15:00,2024-03-03 02:00:00,49457,73.66,32,76,203.75,0.0,0.0,0.0,203.75,Cash,Sun Taxi,41.878865584,-87.625192142,41.980264315,-87.913624596,2024-03-02 12:00:00
8465,721d98019623989e1fc4b338b964f8062c93981b,7805fc9f8d756c8798e3b143635938b447596d082e3113...,2024-03-02 11:30:00,2024-03-02 11:30:00,120,0.00,24,24,200.00,0.0,0.0,0.0,200.00,Cash,Taxi Affiliation Services,41.901206994,-87.676355989,41.901206994,-87.676355989,2024-03-02 11:00:00
9234,5b45815f305c934959bb0ad81e8647881c0cd72e,60bfa66fda3ce439a0ca18ec81dccc1b094f5c97347868...,2024-03-02 10:15:00,2024-03-02 14:15:00,14280,50.70,76,76,168.75,0.0,0.0,4.0,172.75,Cash,Taxi Affiliation Services,41.97907082,-87.903039661,41.97907082,-87.903039661,2024-03-02 10:00:00
9715,93cb24e5a6bc3e50b4fadadc399eae2bf87aa334,24471dbdad65718d90f3f55715fe1c45b75169615d6403...,2024-03-02 09:15:00,2024-03-02 13:00:00,13333,18.86,8,76,106.75,0.0,0.0,0.0,106.75,Cash,City Service,41.89503345,-87.619710672,41.97907082,-87.903039661,2024-03-02 09:00:00
4020,d8ab69390c3e442ab159904621ce20762471caad,1c42c627375b79ab89fd3e55c3ad10309264d758ac270d...,2024-03-02 17:30:00,2024-03-02 18:15:00,2793,41.70,76,72,97.75,0.0,0.0,12.0,109.75,Cash,Chicago Independents,41.980264315,-87.913624596,41.713148612,-87.675075312,2024-03-02 17:00:00
3360,e68e3969bfc2431f20a46c6bee19e3d464ca50ae,763ae7383c3fd4eb95375759ee5d926e3309fa63656a57...,2024-03-02 18:15:00,2024-03-02 19:00:00,3000,2.50,76,72,97.25,0.0,0.0,4.0,101.25,Cash,Taxi Affiliation Services,41.980264315,-87.913624596,41.713148612,-87.675075312,2024-03-02 18:00:00
4303,838e6e2ab45862b9a924a9503c2cbf97e39ea287,8a2802b8001fa243450b71eb66e6e76b4bdc4ea50cbb19...,2024-03-02 17:15:00,2024-03-02 18:15:00,3000,0.00,8,76,95.00,5.0,0.0,0.0,100.00,Credit Card,Taxi Affiliation Services,41.890922026,-87.618868355,41.97907082,-87.903039661,2024-03-02 17:00:00
6321,5f22d9d88dc970e55b0ae4b3f91cddeb3e7c63b4,33f43a33cd31fb3180605dd8e8a1fc7be005a9848fb16e...,2024-03-02 14:30:00,2024-03-02 15:30:00,3087,33.92,76,52,92.63,0.0,0.0,0.0,92.63,Cash,Flash Cab,41.980264315,-87.913624596,41.707311449,-87.534902901,2024-03-02 14:00:00
10762,1e426c0c977c60e4a34a4d0559d5a2273874e021,356744753023949fa63be6a6d7e0aef31f2d50dd3b0258...,2024-03-02 03:30:00,2024-03-02 04:30:00,3161,21.03,52,21,92.63,0.0,0.0,0.0,92.63,Cash,Flash Cab,41.707311449,-87.534902901,41.938666196,-87.711210593,2024-03-02 03:00:00


### Data Modeling

In [54]:
payment_type_master = taxi_trips['payment_type'].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
        'payment_type_id': range(1, len(payment_type_master) + 1),
        'payment_type': payment_type_master
    }
)




In [83]:
company_master = taxi_trips['company'].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
        'company_id': range(1, len(company_master) + 1),
        'company': company_master
    }
)

company_master

,company_id,company
0,1,5 Star Taxi
1,2,City Service
2,3,Flash Cab
3,4,Medallion Leasin
4,5,Tac - Yellow Cab Association
5,6,Taxi Affiliation Services
6,7,Sun Taxi
7,8,Blue Ribbon Taxi Association
8,9,Taxicab Insurance Agency Llc
9,10,Globe Taxi


In [63]:
taxi_trips_id = taxi_trips.merge(payment_type_master, on='payment_type')

taxi_trips_id = taxi_trips_id.merge(company_master, on='company')


In [60]:
taxi_trips_id.drop(['payment_type', 'company'], axis=1, inplace=True)

In [64]:
payment_type_master.to_csv('payment_type_master.csv', index=False)
company_master.to_csv('company_master.csv', index=False)

### Chcking the modeling

#### Payment type

In [65]:
payment_type_master = taxi_trips['payment_type'].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
        'payment_type_id': range(1, len(payment_type_master) + 1),
        'payment_type': payment_type_master
    }
)


In [67]:
new_payment_type_data = [
        {'payment_type': 'Credit Card'},
        {'payment_type': 'x'},
        {'payment_type': 'y'}
]

new_payment_type_mapping = pd.DataFrame(new_payment_type_data)

new_payment_type_mapping

,payment_type
0,Credit Card
1,x
2,y


In [71]:
payment_type_max_id = payment_type_master['payment_type_id'].max()

In [72]:
range(payment_type_max_id +1, payment_type_max_id + len(new_payment_type_mapping) +1)

range(8, 11)

In [73]:
new_payment_type_mapping['payment_type_id'] = range(payment_type_max_id +1, payment_type_max_id + len(new_payment_type_mapping) +1)

new_payment_type_mapping

,payment_type,payment_type_id
0,Credit Card,8
1,x,9
2,y,10


In [77]:
# ~ ugyanaz mint a powershellben a !, az ellentétjét fogja visszaadni a filternek

new_payment_types = new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]

In [78]:
if not new_payment_types.empty:
    payment_type_master = pd.concat([payment_type_master, new_payment_types], ignore_index=True)


In [79]:
payment_type_master

,payment_type_id,payment_type
0,1,Mobile
1,2,Credit Card
2,3,Prcard
3,4,Cash
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,9,x
8,10,y


#### Company

In [ ]:
company_master = taxi_trips['company'].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
        'company_id': range(1, len(company_master) + 1),
        'company': company_master
    }
)

company_master

In [84]:
new_company_data = [
        {'company': 'Setare Inc'},
        {'company': 'x'},
        {'company': 'y'}
]

new_company_mapping = pd.DataFrame(new_company_data)

new_company_mapping

,company
0,Credit Card
1,x
2,y


In [89]:
company_max_id = company_master['company_id'].max()

company_max_id

29

In [90]:
new_company_mapping['company_id'] = range(company_max_id +1, company_max_id + len(new_company_mapping) +1)

new_company_mapping

,company,company_id
0,Credit Card,30
1,x,31
2,y,32


In [91]:
new_companies = new_company_mapping[~new_company_mapping['company'].isin(company_master['company'])]

new_companies

,company,company_id
0,Credit Card,30
1,x,31
2,y,32


In [92]:
if not new_companies.empty:
    company_master = pd.concat([company_master, new_companies], ignore_index=True)


company_master

,company_id,company
0,1,5 Star Taxi
1,2,City Service
2,3,Flash Cab
3,4,Medallion Leasin
4,5,Tac - Yellow Cab Association
5,6,Taxi Affiliation Services
6,7,Sun Taxi
7,8,Blue Ribbon Taxi Association
8,9,Taxicab Insurance Agency Llc
9,10,Globe Taxi
